In [ ]:
import django
import os
import sys
import shutil

path='../../'

source_file_path = f'{path}.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

from dotenv import dotenv_values

_config = dotenv_values(".env")
_username = _config["username"]

session_file_path = f'{path}{_username}.session'

# shutil.copy('Farnoooooood.session', destination_file_path)


project_path = path  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()


In [ ]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [ ]:
from django.db.models import Sum
from django.db.models.functions import TruncMonth
from PostAnalyzer.models import Predict
from asgiref.sync import sync_to_async
# Django ORM equivalent
results = await sync_to_async(
        lambda: list(
            Predict.objects
            .select_related('post__channel')
            .annotate(month=TruncMonth('date'))
            .values('post__channel__name', 'month')
            .annotate(total_profit=Sum('profit'))
            .order_by('month', 'post__channel__name')
        )
    )()

for result in results:
    print(result['post__channel__name'], result['month'], result['total_profit'])

# results


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

# Sample data to simulate results from Django ORM
# Assuming `results` has the fields `channel_name`, `month`, and `total_profit`


# Convert the data to a DataFrame
df = pd.DataFrame(results)

# Pivot the DataFrame to format it for plotting
pivot_df = df.pivot(index='month', columns='post__channel__name', values='total_profit')

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
pivot_df.plot(kind='line', marker='o', ax=ax)

# Adding a high contrast horizontal line at y=0
ax.axhline(y=0, color='red', linestyle='--', linewidth=2)
# ax.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Baseline (y=0)')

# Setting labels, title, and legend
plt.title('Monthly Total Profit by Channel')
plt.xlabel('Month')
plt.ylabel('Total Profit')
plt.legend(title='Channel')
plt.grid()
for channel in pivot_df.columns:
    for month, value in pivot_df[channel].items():
        if pd.notnull(value):  # Only annotate non-null values
            ax.text(month, value, f'{value:.2f}', ha='center', va='bottom', fontsize=8)

plt.show()




In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


# Convert the data to a DataFrame
df = pd.DataFrame(results)
df['month'] = pd.to_datetime(df['month']).dt.strftime('%Y-%m \n(%b)')
# df['month'] = pd.to_datetime(df['month'])

# Sort by month to ensure proper order
df = df.sort_values(by='month')
# Pivot the DataFrame to format it for plotting
pivot_df = df.pivot(index='month', columns='post__channel__name', values='total_profit')

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
pivot_df.plot(kind='bar', ax=ax)

# Adding a high contrast horizontal line at y=0
ax.axhline(y=0, color='red', linestyle='--', linewidth=2)

# Setting labels, title, and legend
plt.title('Monthly Total Profit by Channel')
plt.xlabel('Month')
plt.ylabel('Total Profit')
plt.legend(title='Channel')
plt.grid(axis='y')  # Grid only for y-axis to improve readability

# Adding data labels to each bar
for p in ax.patches:
    height = p.get_height()
    if pd.notnull(height):  # Only annotate non-null values
        ax.text(
            p.get_x() + p.get_width() / 2,
            height,
            f'{height:.2f}',
            ha='center',
            va='bottom',
            fontsize=8
        )

plt.tight_layout()
plt.show()


In [ ]:
from django.db.models import Sum, Count
from PostAnalyzer.models import Predict
from asgiref.sync import sync_to_async

# Define an async function
gross_loss = await sync_to_async(
    lambda: Predict.objects.filter(
        status__name__in=['FAILED WITH PROFIT', 'FAILED']
    ).aggregate(
        failed_profit=Sum('profit'),
        failed_count=Count('id')
    )
)()

gross_loss


In [ ]:
from django.db.models import Sum, Count
from PostAnalyzer.models import Predict
from asgiref.sync import sync_to_async

# Define an async function
gross_win = await sync_to_async(
    lambda: Predict.objects.filter(
        status__name__in=['SUCCESS', 'FULLTARGET']
    ).aggregate(
        win_profit=Sum('profit'),
        win_count=Count('id')
    )
)()

gross_win


In [ ]:
from django.db.models import Sum, Count, Case, When, IntegerField, FloatField
from PostAnalyzer.models import Predict
from asgiref.sync import sync_to_async

# Define an async function
result = await sync_to_async(
    lambda: Predict.objects.filter(**{}).aggregate(
        gross_loss=Sum(
            Case(
                When(status__name__in=['FAILED WITH PROFIT', 'FAILED'], then='profit'),
                output_field=FloatField()
            )
        ),
        loss_count=Count(
            Case(
                When(status__name__in=['FAILED WITH PROFIT', 'FAILED'], then=1),
                output_field=IntegerField()
            )
        ),
        gross_profit=Sum(
            Case(
                When(status__name__in=['SUCCESS', 'FULLTARGET'], then='profit'),
                output_field=FloatField()
            )
        ),
        win_count=Count(
            Case(
                When(status__name__in=['SUCCESS', 'FULLTARGET'], then=1),
                output_field=IntegerField()
            )
        ),
        total_gross=Sum('profit', output_field=FloatField()),  # Total profit for all statuses
        total_count=Count('id', output_field=IntegerField()) 
    )
)()

gross_loss = result['gross_loss']
loss_count = result['loss_count']
gross_profit = result['gross_profit']
win_count = result['win_count']
total_count = result['total_count']
total_gross = result['total_gross']

loss_rate = loss_count/total_count
win_rate = win_count/total_count
average_loss = gross_loss/loss_count
average_win = gross_profit/win_count
expectancy = (win_rate * average_win)-(abs(loss_rate * average_loss))
profit_factor =  abs(gross_profit/gross_loss if gross_loss else 1)
payoff_ratio = abs(average_win/average_loss if average_loss else 1)

print(f'total_count: {total_count}')
print(f'total_gross: {total_gross}')
print(f'total_count: {total_count}')
print(f'gross_loss: {gross_loss}')
print(f'loss_count: {loss_count}')
print(f'gross_profit: {gross_profit}')
print(f'win_count: {win_count}')
print(f'loss_rate: {loss_rate}')
print(f'win_rate: {win_rate}')
print(f'average_loss: {average_loss}')
print(f'average_win: {average_win}')
print(f'expectancy: {expectancy}')
print(f'profit_factor: {profit_factor}')
print(f'payoff_ratio: {payoff_ratio}')


In [ ]:
from django.db.models import Sum, Count
from PostAnalyzer.models import Predict
from asgiref.sync import sync_to_async

# Define an async function to get sum and count of profits for each status
result = await sync_to_async(
    lambda: list(
        Predict.objects.values('status__name')
        .annotate(
            total_profit=Sum('profit'),
            profit_count=Count('id')
        )
        .order_by('status__name')
    )
)()

result


In [ ]:
from asgiref.sync import sync_to_async
from PostAnalyzer.models import TakeProfitTarget
from django.db.models import Count, Max

async def get_tp_num_counts():
    # Step 1: Perform the inner query, grouping by predict_id, counting entries, and getting max profit
    subquery = await sync_to_async(
        lambda: list(
            TakeProfitTarget.objects.filter(active=True)
            .values('predict_id')
            .annotate(
                tp_index=Count('id'),            
                max_profit=Max('profit')        
            )
        )
    )()

    
    tp_result = {}
    for entry in subquery:
        tp_index = entry['tp_index']
        max_profit = entry['max_profit']
        if tp_index not in tp_result:
            tp_result[tp_index] = {'count': 0, 'total_profit': 0}
        tp_result[tp_index]['count'] += 1
        tp_result[tp_index]['total_profit'] += max_profit

    
    result = [
        {'tp_index': tp_index, 'count': data['count'], 'total_profit': data['total_profit']}
        for tp_index, data in sorted(tp_result.items())
    ]
    
    return result

await get_tp_num_counts()


In [25]:
from django.db.models import Count, Case, When, Value, CharField
from PostAnalyzer.models import Channel
from asgiref.sync import sync_to_async

async def x():
    queryset = await sync_to_async(
        lambda: list(
            Channel.objects
            .filter(
                post__predict__status__name__in=['FAILED', 'FAILED WITH PROFIT', 'SUCCESS', 'FULLTARGET']
            )
            .annotate(
                status_group=Case(
                    When(post__predict__status__name__in=['FAILED', 'FAILED WITH PROFIT'], then=Value('FAILED_GROUP')),
                    When(post__predict__status__name__in=['SUCCESS', 'FULLTARGET'], then=Value('SUCCESS_GROUP')),
                    output_field=CharField(),
                ),
                predict_count=Count('post__predict', distinct=True),
            )
            .values('channel_id', 'name', 'status_group', 'predict_count')
            .order_by('channel_id', 'status_group')
        )
    )()
    return queryset

# Call the async function
await x()


[{'channel_id': '1511133584',
  'name': 'RASTAD',
  'status_group': 'FAILED_GROUP',
  'predict_count': 65},
 {'channel_id': '1511133584',
  'name': 'RASTAD',
  'status_group': 'SUCCESS_GROUP',
  'predict_count': 50},
 {'channel_id': '1566206468',
  'name': 'FEYZIAN',
  'status_group': 'FAILED_GROUP',
  'predict_count': 207},
 {'channel_id': '1566206468',
  'name': 'FEYZIAN',
  'status_group': 'SUCCESS_GROUP',
  'predict_count': 84}]

In [37]:
from django.db.models import Count, Case, When, Value, CharField, F
from PostAnalyzer.models import Channel, Post, Predict, PostStatus
from asgiref.sync import sync_to_async

# Define the async function
async def get_channel_data_with_totals():
    # Subquery to get total predictions per channel
    result = await sync_to_async(
        lambda: list(
            Predict.objects
            .filter(post__channel__isnull=False)
            .annotate(channel_id=F('post__channel__channel_id'))
            .values('channel_id')
            .annotate(total_count=Count('id'))
        )
    )()
    
    return result

# Call the async function
channel_data_with_totals = await get_channel_data_with_totals()
print(channel_data_with_totals)




[{'channel_id': '1566206468', 'total_count': 362}, {'channel_id': '1511133584', 'total_count': 117}]


In [42]:
# Input data
status_data = [
    {'channel_id': '1511133584', 'name': 'RASTAD', 'status_group': 'FAILED_GROUP', 'predict_count': 65},
    {'channel_id': '1511133584', 'name': 'RASTAD', 'status_group': 'SUCCESS_GROUP', 'predict_count': 50},
    {'channel_id': '1566206468', 'name': 'FEYZIAN', 'status_group': 'FAILED_GROUP', 'predict_count': 207},
    {'channel_id': '1566206468', 'name': 'FEYZIAN', 'status_group': 'SUCCESS_GROUP', 'predict_count': 84}
]

total_data = [
    {'channel_id': '1566206468', 'total_count': 362},
    {'channel_id': '1511133584', 'total_count': 117}
]

# Initialize a dictionary to hold the results
result = {}

# Process the status data to accumulate counts
for entry in status_data:
    channel_id = entry['channel_id']
    if channel_id not in result:
        result[channel_id] = {
            'channel_id': channel_id,
            'name': entry['name'],  # Capture the name
            'FAILED_GROUP': 0,
            'SUCCESS_GROUP': 0
        }
    
    # Accumulate the counts
    if entry['status_group'] == 'FAILED_GROUP':
        result[channel_id]['FAILED_GROUP'] += entry['predict_count']
    elif entry['status_group'] == 'SUCCESS_GROUP':
        result[channel_id]['SUCCESS_GROUP'] += entry['predict_count']

# Process the total data to combine with the result
for entry in total_data:
    channel_id = entry['channel_id']
    if channel_id in result:
        result[channel_id]['total_count'] = entry['total_count']

# Convert the result to a list
final_result = list(result.values())

# Output the final result
print(final_result)

[{'channel_id': '1511133584', 'name': 'RASTAD', 'FAILED_GROUP': 65, 'SUCCESS_GROUP': 50, 'total_count': 117}, {'channel_id': '1566206468', 'name': 'FEYZIAN', 'FAILED_GROUP': 207, 'SUCCESS_GROUP': 84, 'total_count': 362}]
